In [25]:
from bs4 import BeautifulSoup
import requests
from PIL import Image
from PIL.ImageDraw import Draw
import sys
sys.path.append("../server/")
resp=requests.get('https://www.trianglemtb.com')
soup=BeautifulSoup(resp.text,'html.parser') 

In [26]:
def get_status_dict():
    resp = requests.get('https://www.trianglemtb.com')
    soup=BeautifulSoup(resp.text,'html.parser') 
    status_dict = {}
    a = soup.findAll("span")
    for item in a:
        if 'CLOSED' in item.text or 'OPEN' in item.text:
            test = item
            status_dict[test.parent.parent.parent.a.text] = True if 'OPEN' in item.text else False
    return status_dict

In [27]:
status = get_status_dict()

In [28]:
from PIL import ImageFont
from PIL.ImageDraw import Draw

# The FF SubVario Condensed Medium pixel font.
SUBVARIO_CONDENSED_MEDIUM = {
    'file': 'assets/SubVario-Condensed-Medium.otf',
    'size': 24,
    'height': 20,
    'y_offset': 1,
    'width_overrides': {
        ' ': 5,
        '1': 8
    }
}

# The FF Screenstar Small Regular pixel font.
SCREENSTAR_SMALL_REGULAR = {
    'file': 'assets/Screenstar-Small-Regular.otf',
    'size': 12,
    'height': 10,
    'y_offset': 4,
    'width_overrides': {
        ' ': 3
    }
}


def draw_text(text, font_spec, text_color, xy=None, anchor=None,
              box_color=None, box_padding=0, border_color=None, border_width=0,
              image=None):
    """Draws centered text on an image, optionally in a box."""

    draw = Draw(image)
    text_size = font_spec['size']
    font = ImageFont.truetype(font_spec['file'], size=text_size)

    # Measure the width of each character.
    character_widths = []
    for character in text:
        # Override the measured width, if specified.
        width_overrides = font_spec['width_overrides']
        if character in width_overrides.keys():
            character_width = width_overrides[character]
        else:
            character_width, _ = draw.textsize(character, font)
        character_widths.append(character_width)
    text_width = sum(character_widths)

    # If any xy is specified, use it.
    text_height = font_spec['height']
    if xy:
        x = xy[0] - text_width // 2
        y = xy[1] - text_height // 2

    # If any anchor is specified, adjust the xy.
    if anchor == 'center':
        x = image.width // 2 - text_width // 2
        y = image.height // 2 - text_height // 2
    elif anchor == 'center_x':
        x = image.width // 2 - text_width // 2
    elif anchor == 'center_y':
        y = image.height // 2 - text_height // 2
    elif anchor == 'bottom_right':
        x = image.width - box_padding - border_width - text_width
        y = image.height - box_padding - border_width - text_height

    # Draw the box background and border.
    box_xy = [x - box_padding,
              y - box_padding,
              x + text_width + box_padding,
              y + text_height + box_padding]
    border_xy = [box_xy[0] - border_width,
                 box_xy[1] - border_width,
                 box_xy[2] + border_width,
                 box_xy[3] + border_width]
    if border_color:
        draw.rectangle(border_xy, border_color)
    if box_color:
        draw.rectangle(box_xy, box_color)

    # Draw the text character by character.
    y -= font_spec['y_offset']
    for index in range(len(text)):
        character = text[index]
        draw.text((x, y), character, text_color, font)
        x += character_widths[index]


In [22]:
def image(status):
    
    
    from calendar import Calendar
    from calendar import monthrange
    from calendar import SUNDAY
    from collections import Counter
    from datetime import datetime
    from datetime import timedelta
    from dateutil.parser import parse
    from googleapiclient import discovery
    from googleapiclient.http import build_http
    from logging import warning
    from logging import error
    from oauth2client.client import HttpAccessTokenRefreshError
    from PIL import Image
    from PIL.ImageDraw import Draw
    """Generates an image with MTB status"""

    # The width of the display in pixels.
    DISPLAY_WIDTH = 640

    # The height of the display in pixels.
    DISPLAY_HEIGHT = 384

    # The color of the image background.
    BACKGROUND_COLOR = (255, 255, 255)

    # The color used for days.
    NUMBER_COLOR = (0, 0, 0)

    # The color used for the current day and events.
    TODAY_COLOR = (255, 255, 255)

    # The squircle image file.
    SQUIRCLE_FILE = 'assets/squircle.gif'

    # The dot image file.
    DOT_FILE = 'assets/dot.gif'

    # The offset used to vertically center the numbers in the squircle.
    NUMBER_Y_OFFSET = 1

    # The horizontal margin between dots.
    DOT_MARGIN = 4

    # The vertical offset between dots and numbers.
    DOT_OFFSET = 16

    # The color used to highlight the current day and events.
    HIGHLIGHT_COLOR = (255, 0, 0)

    TEXT_COLOR = (0, 0, 0)

    trail_num = len(status.keys())

    # Create a blank image.
    image = Image.new(mode='RGB', size=(DISPLAY_WIDTH, DISPLAY_HEIGHT),
                      color=BACKGROUND_COLOR)
    draw = Draw(image)

    # Determine the spacing of the days in the image.
    y_stride = DISPLAY_HEIGHT // (trail_num + 1)

    # Draw each week in a row.
    trails = list(status.keys())
    for trail_index in range(trail_num):

        y = (trail_index + 1) * y_stride

        draw_text(status[trails[trail_index]], SUBVARIO_CONDENSED_MEDIUM, TEXT_COLOR,
                  xy=(100, y - NUMBER_Y_OFFSET), image=image)
    return image

In [24]:
im = image(status)

OSError: cannot open resource